In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import numpy as np

import tf_datasets
# import tf_network

In [2]:
# class MyModel(keras.Model):
#     def train_step(self, data):
#         inputs, targets = data
#         trainable_vars = self.trainable_variables
#         with tf.GradientTape() as tape2:
#             with tf.GradientTape() as tape1:
#                 preds = self(inputs, training=True)  # Forward pass
#                 # Compute the loss value
#                 # (the loss function is configured in `compile()`)
#                 loss = self.compiled_loss(targets, preds)
#             # Compute first-order gradients
#             dl_dw = tape1.gradient(loss, trainable_vars)
#         # Compute second-order gradients
#         d2l_dw2 = tape2.gradient(dl_dw, trainable_vars)

#         # Combine first-order and second-order gradients
#         grads = [0.5 * w1 + 0.5 * w2 for (w1, w2) in zip(d2l_dw2, dl_dw)]

#         # Update weights
#         self.optimizer.apply_gradients(zip(grads, trainable_vars))

#         # Update metrics (includes the metric that tracks the loss)
#         self.compiled_metrics.update_state(targets, preds)
#         # Return a dict mapping metric names to current value
#         return {m.name: m.result() for m in self.metrics}

In [3]:


# # Construct an instance of MyModel
# def get_model():
#     inputs = keras.Input(shape=(784,))
#     intermediate = layers.Dense(256, activation="relu")(inputs)
#     outputs = layers.Dense(10, activation="softmax")(intermediate)
#     model = MyModel(inputs, outputs)
#     return model


# # Prepare data
# (x_train, y_train), _ = keras.datasets.mnist.load_data()
# x_train = np.reshape(x_train, (-1, 784)) / 255

# model = get_model()
# model.compile(
#     optimizer=keras.optimizers.SGD(learning_rate=1e-2),
#     loss="sparse_categorical_crossentropy",
#     metrics=["accuracy"],
# )
# model.fit(x_train, y_train, epochs=3, batch_size=1024, validation_split=0.1)

In [4]:
dlayers = [27, 34, 2]
mlayers = [27, 34, 2]

In [5]:
def training_model():
    inputs = keras.layers.Input(27 ,)
    
    y1 = keras.layers.Input(2 ,)
    y2 = keras.layers.Input(2 ,)
    
    x = inputs
    for i in range(1, len(dlayers)-1):
        x = keras.layers.Dense(dlayers[i], activation="tanh", 
                              kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1),
                              bias_initializer=tf.keras.initializers.Constant(0.5))(x)    
        if 1:
            x = keras.layers.BatchNormalization(momentum=0.999, #0.999 
                epsilon=0.0001, # 1e-4
             )(x)
        x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(dlayers[-1], activation="tanh", 
                          kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1),
                          bias_initializer=tf.keras.initializers.Constant(0.5))(x)
    x = keras.layers.BatchNormalization(momentum=0.999, #0.999 
                epsilon=0.0001, # 1e-4
             )(x)
    output1 = x
    
    x = inputs
    for i in range(1, len(mlayers)-1):
        x = keras.layers.Dense(mlayers[i], activation="tanh", 
                              kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1),
                              bias_initializer=tf.keras.initializers.Constant(0.5))(x)    
        if 1:
            x = keras.layers.BatchNormalization(momentum=0.999, #0.999 
                epsilon=0.0001, # 1e-4
             )(x)
        x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(mlayers[-1], activation="tanh", 
                          kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1),
                          bias_initializer=tf.keras.initializers.Constant(0.5))(x)
    x = keras.layers.BatchNormalization(momentum=0.999, #0.999 
                epsilon=0.0001, # 1e-4
             )(x)
    output2 = x
    
    x = inputs
    for i in range(1, len(mlayers)-1):
        x = keras.layers.Dense(mlayers[i], activation="tanh", 
                          kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1),
                          bias_initializer=tf.keras.initializers.Constant(0.5))(x)   
        if 1:
            x = keras.layers.BatchNormalization(momentum=0.999, #0.999 
                epsilon=0.0001, # 1e-4
             )(x)
        x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(mlayers[-1], activation="tanh", 
                          kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1),
                          bias_initializer=tf.keras.initializers.Constant(0.5))(x)
    x = keras.layers.BatchNormalization(momentum=0.999, #0.999 
                epsilon=0.0001, # 1e-4
             )(x)
    output3 = x
        
    model = keras.Model([inputs, y1, y2], [output1, output2, output3])
    return model, y1, y2, output1, output2, output3

In [6]:
model_orig, y1, y2, out1, out2, out3 = training_model()

In [14]:
# import tensorflow as tf
# tf.config.run_functions_eagerly(True)

class SAMModel(tf.keras.Model):
    def __init__(self, model, rho=0.05):
        """
        p, q = 2 for optimal results as suggested in the paper
        (Section 2)
        """
        super(SAMModel, self).__init__()
        self.model = model
        self.rho = rho
        
#     def call(self, data):
#         return self.model(data)
#         with tf.GradientTape() as tape:
#             predictions = self.model(data)
#             loss = self.compiled_loss((data, predictions), predictions)
#         return loss

    def train_step(self, data):
        inputs = data
        (inp, y1, y2) = data[0]
        labels = y1
        e_ws = []
        with tf.GradientTape() as tape:
            predictions = self.model(data)
            [out1, out2, out3] = predictions 
            
            loss_detector = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y1, logits=out1))
            loss_modifier = tf.nn.l2_loss(out2 - y2)

            loss_modifier_mve = 0.5*tf.reduce_sum(((out2 - y2)**2)/(out3**2 + 1e-4)) + 0.5*tf.reduce_sum(tf.math.log(out3**2 + 1e-4)) # mean variance estimate

            loss = loss_detector + loss_modifier
            loss_mve = loss_detector + loss_modifier_mve
            loss = loss + loss_mve
        
        trainable_params = self.model.trainable_variables
        gradients = tape.gradient(loss, trainable_params)
        grad_norm = self._grad_norm(gradients)
        scale = self.rho / (grad_norm + 1e-12)

        for (grad, param) in zip(gradients, trainable_params):
            e_w = grad * scale
            param.assign_add(e_w)
            e_ws.append(e_w)

        with tf.GradientTape() as tape:
            predictions = self.model(data)
            [out1, out2, out3] = predictions 
            
            loss_detector = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y1, logits=out1))
            loss_modifier = tf.nn.l2_loss(out2 - y2)

            loss_modifier_mve = 0.5*tf.reduce_sum(((out2 - y2)**2)/(out3**2 + 1e-4)) + 0.5*tf.reduce_sum(tf.math.log(out3**2 + 1e-4)) # mean variance estimate

            loss = loss_detector + loss_modifier
            loss_mve = loss_detector + loss_modifier_mve
            loss = loss + loss_mve
        
        sam_gradients = tape.gradient(loss, trainable_params)
        for (param, e_w) in zip(trainable_params, e_ws):
            param.assign_sub(e_w)
        
        self.optimizer.apply_gradients(
            zip(sam_gradients, trainable_params))
        
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        (inp, y1, y2) = data[0]
        predictions = self.model(data, training=False)
        [out1, out2, out3] = predictions 
    
        loss_detector = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y1, logits=out1))
        loss_modifier = tf.nn.l2_loss(out2 - y2)

        loss_modifier_mve = 0.5*tf.reduce_sum(((out2 - y2)**2)/(out3**2 + 1e-4)) + 0.5*tf.reduce_sum(tf.math.log(out3**2 + 1e-4)) # mean variance estimate

        loss = loss_detector + loss_modifier
        loss_mve = loss_detector + loss_modifier_mve

        loss = loss + loss_mve
        
#         loss = self.compiled_loss(labels, predictions)
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def _grad_norm(self, gradients):
        norm = tf.norm(
            tf.stack([
                tf.norm(grad) for grad in gradients if grad is not None
            ])
        )
        return norm

In [15]:
model = SAMModel(model_orig)

In [16]:
loss_detector = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y1, logits=out1))
loss_modifier = tf.nn.l2_loss(out2 - y2)

loss_modifier_mve = 0.5*tf.reduce_sum(((out2 - y2)**2)/(out3**2 + 1e-4)) + 0.5*tf.reduce_sum(tf.math.log(out3**2 + 1e-4)) # mean variance estimate

loss = loss_detector + loss_modifier
model.add_loss(loss)

loss_mve = loss_detector + loss_modifier_mve
model.add_loss(loss_mve)

In [17]:
opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt
             ,run_eagerly=True
             )

In [18]:
import tf_datasets

data_sets, N_tuple = tf_datasets.read_data_sets(dirs=sorted(["../training_data/Original/tdata/"]), use_softmax=True, frac_test=0.25)
scale = { i: max(abs(data_sets.train.get_data()[i].min()), abs(data_sets.train.get_data()[i].max())) for i in data_sets.train.get_data() }

train_data = data_sets.train.get_data()

In [19]:
train_data["inputs"].shape

(6282, 27)

In [20]:
history = model.fit([train_data["inputs"], train_data["labels"], train_data["modvel"]/scale["modvel"]], batch_size=5000, epochs=100, verbose=1
                   )

Epoch 1/100
2/2 [==============================] - 2s 22ms/step
Epoch 2/100
2/2 [==============================] - 0s 9ms/step
Epoch 3/100
2/2 [==============================] - 0s 8ms/step
Epoch 4/100
2/2 [==============================] - 0s 10ms/step
Epoch 5/100
2/2 [==============================] - 0s 18ms/step
Epoch 6/100
2/2 [==============================] - 0s 8ms/step
Epoch 7/100
2/2 [==============================] - 0s 9ms/step
Epoch 8/100
2/2 [==============================] - 0s 7ms/step
Epoch 9/100
2/2 [==============================] - 0s 8ms/step
Epoch 10/100
2/2 [==============================] - 0s 10ms/step
Epoch 11/100
2/2 [==============================] - 0s 14ms/step
Epoch 12/100
2/2 [==============================] - 0s 9ms/step
Epoch 13/100
2/2 [==============================] - 0s 12ms/step
Epoch 14/100
2/2 [==============================] - 0s 12ms/step
Epoch 15/100
2/2 [==============================] - 0s 9ms/step
Epoch 16/100
2/2 [========================